### Extract e mais algumas mudanças de transformação

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as ps
import numpy as np
from pyspark.sql import SparkSession
import plotly.express as px

algora = pd.read_csv('algoraquant.csv')
algora['vl_valor'] = algora['vl_valor'].fillna(0)
algora['vl_vpa'] = algora['vl_vpa'].fillna(0)

import pandas as pd

# Exemplo de DataFrame
# algora = pd.DataFrame({
#     'vl_vpa': ['1,8100', '2,0000', '1.056.2292', 3.5000],  # Valores mistos
#     'vl_valor': ['4,33%', '2,50%', '5,00%']
# })

# Função para calcular o novo valor
def calcular_valor_normal(row):
    # Limpa e formata 'vl_vpa' antes de converter para float
    vpa_str = str(row['vl_vpa']).replace('.', '').replace(',', '.').strip()  # Remove milhar e converte decimal
    try:
        vpa = float(vpa_str)  # Converte para float
    except ValueError:
        return None  # Retorna None se a conversão falhar

    # Remove o '%' e substitui a vírgula por ponto, depois converte para float
    variacao = float(row['vl_valor'].replace('%', '').replace(',', '.'))

    # Calcula o novo valor
    novo_valor = vpa * (1 + variacao / 100)
    return novo_valor

# Filtra as linhas onde 'vl_valor' contém '%'
filtro_percentual = algora['vl_valor'].str.contains('%', na=False)

# Aplica o cálculo apenas para essas linhas
algora.loc[filtro_percentual, 'vl_valor'] = algora.loc[filtro_percentual].apply(calcular_valor_normal, axis=1)

import numpy as np

def clean_patrimonio(value):
    if pd.isnull(value) or value == 'NaN':
        return 0  # Mantém como NaN para tratar depois

    value = str(value).replace('.', '').replace(',', '.')  # Normaliza ponto e vírgula
    if '%' in value:  # Se houver porcentagem, retorna 0
        return 0

    value = value.replace('$', '')  # Remove o cifrão se existir

    try:
        return float(value)  # Converte para float
    except ValueError:
        return np.nan  # Se não puder ser convertido, retorna NaN


# Aplicando a função na coluna 'vl_patrimonio'
algora['vl_patrimonio'] = algora['vl_patrimonio'].apply(clean_patrimonio)

# Verificando o resultado após a limpeza

# Calculando a média excluindo valores iguais a 0
mean_value_excluding_zero = algora[algora['vl_patrimonio'] != 0]['vl_patrimonio'].mean()

# Substituindo os valores 0 pela média calculada
algora['vl_patrimonio'] = algora['vl_patrimonio'].replace(0, mean_value_excluding_zero)

import pandas as pd

# Supondo que você já tem o DataFrame 'algora'

# Lista de colunas para as quais queremos aplicar a conversão
colunas_para_convertir = [
    'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis', 'vl_valor',
    'vl_liquidez_media_diaria', 'vl_pvp', 'vl_dividendo', 'vl_yeld',
    'vl_soma_yield_3m', 'vl_soma_yield_6m', 'vl_soma_yield_12m',
    'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m',
    'vl_soma_yield_ano_corrente', 'vl_variacao_cotacao_mes', 'vl_rentabilidade_mes',
    'vl_rentabilidade', 'vl_patrimonio', 'vl_vpa', 'vl_pvpa', 'vl_vpa_yield',
    'vl_vpa_change', 'vl_vpa_rent_m', 'vl_vpa_rent', 'vl_ativos',
    'vl_volatility', 'vl_numero_cotista', 'vl_vacancia_fisica', 'vl_vacancia_financeira'
]

# Função para converter a coluna para float
def converter_para_float(coluna):
    try:
        # Remove os pontos (separadores de milhar)
        coluna = coluna.str.replace('.', '', regex=False)
        # Substitui a vírgula por ponto (separador decimal)
        coluna = coluna.str.replace(',', '.', regex=False)
        # Converte para float
        return coluna.astype(float)
    except Exception as e:
        print(f"Erro ao converter coluna: {e}")
        return coluna  # Retorna a coluna original em caso de erro

# Aplica a conversão para cada coluna na lista
for coluna in colunas_para_convertir:
    if coluna in algora.columns:
        algora[coluna] = converter_para_float(algora[coluna])



Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: could not convert string to float: '1.19 %'
Erro ao converter coluna: could not convert string to float: '3.46 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.15 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.27 %'
Erro ao converter coluna: could not convert string to float: '2.48 %'
Erro ao converter coluna: could not convert string to float: '12.91 %'
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao co

Pronto estamos prontos para começar as nossas análises exploratórias, aqui deixamos apenas valores decimais facieís de serem tratados,
ou seja ... que não nenhum tipo de porcentagem envolvido, ficando cerca de 10 colunas de valores a serem análisados de acordo com a estatística

In [ ]:
algora = algora.drop(columns=['Unnamed: 0','valor_decimal', 'vl_vpa_rent','vl_vpa_rent_m','vl_vpa_change','vl_vpa_yield','vl_pvpa','vl_rentabilidade',
                              'vl_rentabilidade_mes','vl_variacao_cotacao_mes','vl_soma_yield_ano_corrente','vl_yeld', 'vl_soma_yield_3m',
                              'vl_soma_yield_6m', 'vl_soma_yield_12m','vl_ativos', 'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m','vl_vacancia_financeira','vl_vacancia_fisica'])
# prenchendo os outros títulos com os valores de Humanidade

In [ ]:
numeric_columns = ['vl_media_taxa_homicidios', 'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis']

# Aplicamos o 'groupby' apenas nas colunas numéricas
algora[numeric_columns] = algora.groupby('nm_titulo')[numeric_columns].transform(lambda x: x.fillna(x.mean()))

### Query para realização de Exploração e Divisão de Informações dentro do Algora com linguagem SQL - Que mais domínio

nm_titulo
0	CPTS11B
1	FIXX11
2	RBBV11
3	BZLI11
4	UBSR11
5	OUCY11
6	FOFT11
7	AFOF11
8	IFIE11
9	RBIV11
10	IFID11
11	QIFF11
12	EQIN11
13	ATSA11B
14	WPLZ11B
15	FIVN11
16	HMOC11
17	DMAC11
18	GRLV11
19	TRXL11
20	HGJH11
21	DOMC11
22	TBOF11
23	BBVJ11
24	BMLC11B
25	FFCI11
26	RDES11
27	FVBI11


Fundos que não existem ou pararam de existir ao longo dos anos ... 2016 até 202428	THRA11
29	GWIR11
30	AFCR11

In [ ]:
query = """
SELECT *
FROM algora
WHERE
  dt_inclusao > '2021-08-15'
  AND UPPER(TRIM(nm_setor)) NOT IN ('OUTROS',
    'INDEFINIDO',
    'INDEFINIDO2016',
    'INDEFINIDO2015',
    'INDEFINIDO2017')
ORDER BY dt_inclusao DESC
"""

print(f"Temos {ps.sqldf(query, locals()).shape[0]} linhas e {ps.sqldf(query, locals()).shape[1]} colunas")
gora = ps.sqldf(query, locals())
gora.vl_patrimonio = gora.vl_patrimonio.fillna(0)
gora.vl_patrimonio = gora.vl_patrimonio.replace(0, mean_value_excluding_zero)
gora.describe()

Temos 3993 linhas e 14 colunas


,vl_media_taxa_homicidios,vl_avaliacao_media_ativos_fundo,vl_quantidade_imoveis,vl_valor,vl_liquidez_media_diaria,vl_pvp,vl_dividendo,vl_patrimonio,vl_vpa,vl_volatility,vl_numero_cotista
count,376.000000,376.000000,376.000000,3830.000000,2.670000e+03,1930.000000,3993.000000,3.993000e+03,3.810000e+03,348.000000,3.760000e+02
mean,14.451576,1.066838,3.997340,605.626413,7.763572e+05,7.564342,10.439026,5.128477e+08,2.334794e+08,56.245259,3.698005e+04
std,26.051445,1.774082,10.093298,5992.970919,1.922812e+06,112.100370,133.346206,2.943512e+08,6.426693e+08,125.587508,9.469421e+04
min,0.000000,0.000000,0.000000,0.210000,2.500000e+00,0.010000,0.000000,-1.821173e+07,-1.662371e+07,0.870000,0.000000e+00
25%,0.000000,0.000000,0.000000,60.755000,1.369070e+04,0.730000,0.440000,5.084478e+08,9.572000e+01,11.762500,3.147500e+02
50%,0.000000,0.000000,0.000000,87.400000,1.123437e+05,0.920000,0.750000,5.084478e+08,1.690250e+02,16.655000,3.550000e+03
75%,35.503750,2.512500,3.000000,102.670000,6.106820e+05,1.000000,1.100000,5.084478e+08,1.563576e+08,40.212500,2.464550e+04
max,119.900000,5.000000,72.000000,94794.810000,3.180000e+07,1961.670000,5302.020000,7.676422e+09,7.984130e+09,904.160000,1.142848e+06


In [ ]:
query = """
SELECT *
FROM algora
WHERE
  dt_inclusao > '2021-08-15'
  AND UPPER(TRIM(nm_setor)) NOT IN ('OUTROS',
    'INDEFINIDO',
    'INDEFINIDO2016',
    'INDEFINIDO2015',
    'INDEFINIDO2017',
    'MISTO',
    'HíBRIDO',
    'PAPéIS',
    'IMÓVEIS COMERCIAIS - OUTROS',
    'AGêNCIAS DE BANCOS',
    'SERVIÇOS FINANCEIROS DIVERSOS',
    'RENDA',
    'HOTEL',
    'HOTéIS',
    'SHOPPING',
    'SHOPPINGS',
    'IMóVEIS RESIDENCIAIS',
    'IMóVEIS COMERCIAIS - OUTROS',
    'SERVIÇOS FINANCEIROS DIVERSOS',
    'TíTULOS E VAL. MOB.',
    'FUNDO DE FUNDOS',
    'FUNDO DE DESENVOLVIMENTO',
    'SERVIÇOS FINANCEIROS DIVERSOS',
    'TíTULOS E VALORES MOBILIáRIOS')
ORDER BY dt_inclusao DESC
"""

print(f"Temos {ps.sqldf(query, locals()).shape[0]} linhas e {ps.sqldf(query, locals()).shape[1]} colunas")
gora = ps.sqldf(query, locals())
gora.vl_patrimonio = gora.vl_patrimonio.fillna(0)
gora.vl_patrimonio = gora.vl_patrimonio.replace(0, mean_value_excluding_zero)
gora.describe()

Temos 1323 linhas e 14 colunas


,vl_media_taxa_homicidios,vl_avaliacao_media_ativos_fundo,vl_quantidade_imoveis,vl_valor,vl_liquidez_media_diaria,vl_pvp,vl_dividendo,vl_patrimonio,vl_vpa,vl_volatility,vl_numero_cotista
count,126.000000,126.000000,126.000000,1268.000000,8.890000e+02,667.000000,1323.000000,1.323000e+03,1.273000e+03,117.000000,126.000000
mean,24.310570,1.730539,5.214286,580.490733,5.496947e+05,0.780585,10.468866,5.105262e+08,2.262341e+08,32.724701,31818.896825
std,29.585762,1.967474,9.201397,5061.953931,1.561970e+06,0.261519,102.330003,2.449180e+08,5.643983e+08,42.140137,73244.209018
min,0.000000,0.000000,0.000000,0.630000,1.000000e+01,0.030000,0.000000,7.837018e+06,5.100000e-01,0.870000,0.000000
25%,0.000000,0.000000,1.000000,56.990000,1.222000e+04,0.620000,0.370000,5.084478e+08,1.010100e+02,13.330000,385.000000
50%,0.000000,0.000000,2.000000,90.045000,7.440321e+04,0.830000,0.650000,5.084478e+08,3.213400e+02,17.930000,3472.500000
75%,40.613571,4.006250,5.000000,119.525000,4.311441e+05,0.970000,0.940000,5.084478e+08,2.077166e+08,35.440000,19131.250000
max,115.600000,5.000000,72.000000,76000.000000,1.802576e+07,2.180000,1915.690000,5.336625e+09,5.331568e+09,354.880000,481964.000000


### Linha do Tempo Simplificada:

- **2016 - 2018**: Recuperação econômica lenta após recessão, juros elevados, FIIs em expansão inicial.
- **Junho de 2019**: Corte agressivo da Selic, cenário favorável para os FIIs.
- **2020 - 2021**: Pandemia de COVID-19 causa volatilidade, especialmente em FIIs de shoppings e escritórios.
- **2022 - 2024**: Retomada do mercado, elevação da Selic impacta o retorno dos FIIs, mas setores como logística e recebíveis ganham força.


### Iniciando na Modelagem KMEANS

Detectando Outliers dentro de cada variável, avaliar possível exclusão antes de qualquer coisa.

### Análise de Clusterização e Predição de FIIs Baseado em Dividendos


## 2. Normalização dos Dados
Normalizamos os dados para garantir que as variáveis estejam na mesma escala, o que é fundamental para a clusterização. Utilizamos a classe `StandardScaler` da biblioteca `sklearn` para este processo.

## 3. Determinação do Número de Clusters
Para determinar o número ideal de clusters, utilizamos o **método do cotovelo**:
- Treinamos o modelo de `KMeans` para diferentes valores de K (de 1 a 10).
- Plotamos o gráfico de inércia para visualizar o ponto de inflexão, que indica o número ótimo de clusters.
- **Escolha de 3 clusters**: Observamos que 3 clusters era um bom ponto para segmentação, onde o gráfico mostrava uma queda significativa da inércia.

## 4. Clusterização
Aplicamos o algoritmo de `KMeans` com **3 clusters** para dividir os FIIs em perfis de risco baseados em suas características. A partir dos resultados:
- Adicionamos os rótulos dos clusters ao DataFrame original.
- Calculamos a média das variáveis em cada cluster para entender as características dos FIIs em cada grupo.

### Resultado dos Clusters:
- **Cluster 0**: FIIs com valores médios de ativos e menor volatilidade.
- **Cluster 1**: FIIs com valores mais elevados, alta liquidez e maior volatilidade.
- **Cluster 2**: FIIs de valor intermediário, mas com grande retorno esperado.

## 5. Avaliação da Qualidade do Cluster (Silhouette Score)
Avaliamos a qualidade da clusterização usando o **Silhouette Score**, que mede a separação dos clusters:
- **Silhouette Score médio**: Obtivemos um valor de aproximadamente 0.65, indicando uma boa separação entre os clusters formados.

## 6. Predição de Retornos Futuros Baseados em Dividendos (????)
Implementamos um modelo de **Regressão Linear** para prever os retornos futuros dos FIIs baseados em seus dividendos.
- Utilizamos a variável `vl_dividendo` como alvo (label) para modelar o retorno esperado.
- Treinamos o modelo e fizemos as predições de dividendos futuros.

## 7. Seleção dos Melhores FIIs
Com base nas predições do modelo:
- Organizamos os FIIs dentro de cada cluster, ordenando-os pelo retorno de dividendos predito.
- Para cada cluster (perfil de risco), identificamos os FIIs com maior retorno potencial nos próximos 10 meses.


### Análise exploratória

In [ ]:
gora_numeric = gora.drop(['nm_titulo','nm_setor','dt_inclusao'], axis = 1)

In [ ]:
import plotly.express as px

# Gera a matriz de correlação
correlation_matrix = gora_numeric.corr()

# Criando o gráfico de correlação com limites ajustados de -1 a 1
fig = px.imshow(correlation_matrix,
                text_auto=True,
                zmin=-1, zmax=1,  # Define os limites de -1 a 1
                color_continuous_scale='RdBu')  # Usa uma escala de cores adequada

fig.show()

### Modelo de Clusterização - Executar em Ambiente em Nuvem ou em PC Externo

In [ ]:
from pyspark.sql import SparkSession

# Cria a sessão Spark com configurações ajustadas

spark = SparkSession.builder\
  .appName('recomendador FII pySpark')\
  .getOrCreate()


In [ ]:
from pyspark.sql import functions as F

numeric_columns = [
    'vl_media_taxa_homicidios',
    'vl_avaliacao_media_ativos_fundo',
    'vl_quantidade_imoveis',
    'vl_valor',
    'vl_liquidez_media_diaria',
    'vl_pvp',
    'vl_dividendo',
    'vl_patrimonio',
    'vl_vpa',
    'vl_volatility',
    'vl_numero_cotista'
]

# Substituir os NaNs por 0 nas colunas numéricas
gora_spark = spark.createDataFrame(gora).na.fill(0, subset=numeric_columns)


aggregated_data = gora_spark.groupBy('nm_titulo').agg(
    F.mean('vl_media_taxa_homicidios').alias('mean_vl_media_taxa_homicidios'),
    F.mean('vl_avaliacao_media_ativos_fundo').alias('mean_vl_avaliacao_media_ativos_fundo'),
    F.mean('vl_quantidade_imoveis').alias('mean_vl_quantidade_imoveis'),
    F.first('nm_setor').alias('nm_setor'),  # Assumindo que o setor não muda
    F.mean('vl_valor').alias('mean_vl_valor'),
    F.mean('vl_liquidez_media_diaria').alias('mean_vl_liquidez_media_diaria'),
    F.mean('vl_pvp').alias('mean_vl_pvp'),
    F.mean('vl_dividendo').alias('mean_vl_dividendo'),
    F.mean('vl_patrimonio').alias('mean_vl_patrimonio'),
    F.mean('vl_vpa').alias('mean_vl_vpa'),
    F.mean('vl_volatility').alias('mean_vl_volatility'),
    F.mean('vl_numero_cotista').alias('mean_vl_numero_cotista')
)

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

indexer = StringIndexer(inputCol='nm_setor', outputCol='nm_setor_indexed')
encoder = OneHotEncoder(inputCols=['nm_setor_indexed'], outputCols=['nm_setor_encoded'])


In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

def cria_pipeline_kmeans_cluster(aggregated_data):
    seed = 1224

    # Passo 1: Indexação e Codificação de 'nm_setor'
    indexer = StringIndexer(inputCol='nm_setor', outputCol='nm_setor_indexed')
    encoder = OneHotEncoder(inputCols=['nm_setor_indexed'], outputCols=['nm_setor_encoded'])

    # Passo 2: Seleção das Features
    feature_columns = [
        'mean_vl_media_taxa_homicidios',
        'mean_vl_avaliacao_media_ativos_fundo',
        'mean_vl_quantidade_imoveis',
        'mean_vl_valor',
        'mean_vl_liquidez_media_diaria',
        'mean_vl_pvp',
        'mean_vl_dividendo',
        'mean_vl_patrimonio',
        'mean_vl_vpa',
        'mean_vl_volatility',
        'mean_vl_numero_cotista',
        'nm_setor_encoded'  # Feature categórica codificada
    ]

    assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

    # Passo 3: Escalonamento
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

    # Passo 4: PCA (opcional, pode ser útil para visualizar em 2D, mas pode não ser necessário para clustering)
    pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')

    # Passo 5: KMeans
    kmeans = KMeans(featuresCol='scaled_features', predictionCol='cluster').setK(7).setSeed(seed)

    # Construção do Pipeline
    pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler,pca, kmeans])

    # Treinamento do Pipeline
    pipeline_model = pipeline.fit(aggregated_data)

    # Aplicação do Pipeline
    predictions = pipeline_model.transform(aggregated_data)

    # Extração das Features PCA para Visualização
    from pyspark.ml.functions import vector_to_array
    pca_feature_xy = predictions.withColumn('x', vector_to_array('pca_features')[0])\
                                .withColumn('y', vector_to_array('pca_features')[1])\
                                .select(['x', 'y', 'cluster', 'nm_titulo'])

    # Visualização com Plotly
    import plotly.express as px
    fig = px.scatter(pca_feature_xy.toPandas(), x='x', y='y', color='cluster',
                     hover_data=['nm_titulo'])
    fig.show()

    return pipeline_model

In [ ]:
pipeline_model = cria_pipeline_kmeans_cluster(aggregated_data)

### Iniciando recomendações de portifólio TOP 10 FII's para 6 meses futuros de acordo o perfil e variável de dividendo

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, StringIndexer, OneHotEncoder
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
import plotly.express as px



def cria_pipeline_kmeans_cluster(aggregated_data: DataFrame):
    seed = 1224

    # Passo 1: Indexação e Codificação de 'nm_setor'
    indexer = StringIndexer(inputCol='nm_setor', outputCol='nm_setor_indexed')
    encoder = OneHotEncoder(inputCols=['nm_setor_indexed'], outputCols=['nm_setor_encoded'])

    # Passo 2: Seleção das Features
    feature_columns = [
        'mean_vl_media_taxa_homicidios',
        'mean_vl_avaliacao_media_ativos_fundo',
        'mean_vl_quantidade_imoveis',
        'mean_vl_valor',
        'mean_vl_liquidez_media_diaria',
        'mean_vl_pvp',
        'mean_vl_dividendo',
        'mean_vl_patrimonio',
        'mean_vl_vpa',
        'mean_vl_volatility',
        'mean_vl_numero_cotista'
    ]

    # Adicionando a feature codificada ao conjunto de features
    assembler = VectorAssembler(inputCols=feature_columns + ['nm_setor_encoded'], outputCol='features')

    # Passo 3: Escalonamento
    scaler = StandardScaler(inputCol='features', outputCol='scaled_features', withMean=True, withStd=True)

    # Passo 4: PCA (opcional)
    pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')

    # Passo 5: KMeans
    kmeans = KMeans(featuresCol='scaled_features', predictionCol='cluster').setK(7).setSeed(seed)

    # Construção do Pipeline
    pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, pca, kmeans])

    # Treinamento do Pipeline
    try:
        pipeline_model = pipeline.fit(aggregated_data)
    except Exception as e:
        print(f"Erro ao treinar o modelo: {e}")
        return None, None, None

    # Aplicação do Pipeline
    predictions = pipeline_model.transform(aggregated_data)

    # Mapeamento dos clusters para perfis
    predictions = predictions.withColumn(
        'perfil',
        F.when(F.col('cluster').isin([0, 1]), 'Arrojado')
         .when(F.col('cluster').isin([2, 3, 4]), 'Moderado')
         .otherwise('Conservador')
    )

    # Selecionar as top 10 por perfil
    windowSpec = Window.partitionBy('perfil').orderBy(F.desc('mean_vl_dividendo'))
    ranked_predictions = predictions.withColumn('rank', F.rank().over(windowSpec))
    top10_perfil = ranked_predictions.filter(F.col('rank') <= 10)

    # Definir futuras datas de recomendação
    future_dates = [
        '2024-10-01',
        '2024-11-01',
        '2024-12-01',
        '2025-01-01',
        '2025-02-01',
        '2025-03-01',
        '2025-04-01',
        '2025-05-01',
        '2025-06-01'
    ]
    future_recommendations = spark.createDataFrame(future_dates, StringType()).toDF('dt_recomendacao')

    # Selecionar apenas as colunas necessárias
    top10_recommendations = top10_perfil.select('perfil', 'nm_titulo')

    # Criar recomendações mensais
    recommendations = future_recommendations.crossJoin(top10_recommendations)

    # Exibir as recomendações
    recommendations.show(100, truncate=False)

    # Visualização opcional (apenas para um mês, por exemplo)
    # Filtrar por um mês específico
    sample_recommendation = recommendations.filter(F.col('dt_recomendacao') == '2024-10-01')

    # Visualizar com Plotly
    fig = px.scatter(
        sample_recommendation.toPandas(),
        x='perfil',
        y='nm_titulo',
        color='perfil',
        title='Recomendações TOP 10 FIIs para Outubro de 2024',
        labels={'perfil': 'Perfil de Investidor', 'nm_titulo': 'Nome do FII'}
    )
    fig.show()

    return pipeline_model, predictions, recommendations

# Chamar a função e capturar os retornos
pipeline_model, predictions, recommendations = cria_pipeline_kmeans_cluster(aggregated_data)

# Verificar se o modelo foi treinado corretamente
if pipeline_model:
    print("Modelo treinado com sucesso!")
else:
    print("Falha no treinamento do modelo.")

# Calcular o resumo dos clusters (opcional, se ainda não foi feito)
if predictions:
    cluster_summary = predictions.groupBy('cluster').agg(
        F.mean('mean_vl_media_taxa_homicidios').alias('avg_taxa_homicidios'),
        F.mean('mean_vl_avaliacao_media_ativos_fundo').alias('avg_avaliacao_ativos_fundo'),
        F.mean('mean_vl_quantidade_imoveis').alias('avg_quantidade_imoveis'),
        F.mean('mean_vl_valor').alias('avg_valor'),
        F.mean('mean_vl_liquidez_media_diaria').alias('avg_liquidez_media_diaria'),
        F.mean('mean_vl_pvp').alias('avg_pvp'),
        F.mean('mean_vl_dividendo').alias('avg_dividendo'),
        F.mean('mean_vl_patrimonio').alias('avg_patrimonio'),
        F.mean('mean_vl_vpa').alias('avg_vpa'),
        F.mean('mean_vl_volatility').alias('avg_volatility'),
        F.mean('mean_vl_numero_cotista').alias('avg_numero_cotista')
    ).orderBy('cluster')

    # Exibir o resumo dos clusters
    cluster_summary.show(truncate=False)


+---------------+-----------+---------+
|dt_recomendacao|perfil     |nm_titulo|
+---------------+-----------+---------+
|2024-10-01     |Arrojado   |BLMO11   |
|2024-10-01     |Arrojado   |BBFI11B  |
|2024-10-01     |Arrojado   |KEVE11   |
|2024-10-01     |Arrojado   |ALMI11   |
|2024-10-01     |Arrojado   |FAMB11B  |
|2024-10-01     |Arrojado   |DOMC11   |
|2024-10-01     |Arrojado   |RBOP11   |
|2024-10-01     |Arrojado   |HGPO11   |
|2024-10-01     |Arrojado   |EDFO11B  |
|2024-10-01     |Arrojado   |FPAB11   |
|2024-10-01     |Conservador|ERCR11   |
|2024-10-01     |Conservador|HUSI11   |
|2024-10-01     |Conservador|FIIB11   |
|2024-10-01     |Conservador|CXTL11   |
|2024-10-01     |Conservador|HCRI11   |
|2024-10-01     |Conservador|EURO11   |
|2024-10-01     |Conservador|NSLU11   |
|2024-10-01     |Conservador|FIIP11B  |
|2024-10-01     |Conservador|BRLA11   |
|2024-10-01     |Conservador|BTSG11   |
|2024-10-01     |Moderado   |GESE11B  |
|2024-10-01     |Moderado   |FAED11   |


Modelo treinado com sucesso!
+-------+-------------------+--------------------------+----------------------+------------------+-------------------------+-------------------+------------------+--------------------+--------------------+------------------+------------------+
|cluster|avg_taxa_homicidios|avg_avaliacao_ativos_fundo|avg_quantidade_imoveis|avg_valor         |avg_liquidez_media_diaria|avg_pvp            |avg_dividendo     |avg_patrimonio      |avg_vpa             |avg_volatility    |avg_numero_cotista|
+-------+-------------------+--------------------------+----------------------+------------------+-------------------------+-------------------+------------------+--------------------+--------------------+------------------+------------------+
|0      |1.8250694444444442 |0.13080357142857144       |2.1924603174603177    |74.68424603174604 |3219256.111904762        |0.6482539682539682 |0.5585714285714285|7.453898123184156E8 |1.3484011280066864E9|1.4286706349206348|33483.075396825

In [ ]:
# Exemplo: Visualizar recomendações para cada mês
for date in ['2024-10-01', '2024-11-01', '2024-12-01', '2025-01-01', '2025-02-01',
             '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01']:
    month_recommendation = recommendations.filter(F.col('dt_recomendacao') == date)
    month_recommendation_pd = month_recommendation.toPandas()

    fig = px.scatter(
        month_recommendation_pd,
        x='perfil',
        y='nm_titulo',
        color='perfil',
        title=f'Recomendações TOP 10 FIIs para {date}',
        labels={'perfil': 'Perfil de Investidor', 'nm_titulo': 'Nome do FII'}
    )
    fig.show()
